In [5]:
import os
import pandas as pd

Setting up directories and imnporting data

In [6]:
# Set up base path to training data directory
training_path = os.path.join(os.getcwd(), 'data', 'training')

# Set up paths to all training CSV files
training_calendar_path = os.path.join(training_path, 'calendar.csv')
training_product_details_path = os.path.join(training_path, 'product_details_train.csv')
training_promotion_details_path = os.path.join(training_path, 'promotion_details_train.csv')
training_transactions_path = os.path.join(training_path, 'transactions_train.csv')
training_weather_path = os.path.join(training_path, 'weather_data.csv')

# Set up base path to forecasting data directory
forecast_path = os.path.join(os.getcwd(), 'data', 'forecasting')

# Set up paths to all 4 forecasting CSV files
forecast_calendar_path = os.path.join(forecast_path, 'forecast_calendar.csv')
forecast_product_details_path = os.path.join(forecast_path, 'forecast_product_details.csv')
forecast_promotion_details_path = os.path.join(forecast_path, 'forecast_promotion_details.csv')
forecast_transactions_path = os.path.join(forecast_path, 'forecast_transactions.csv')



In [7]:
# Importing training data
df_calendar_train = pd.read_csv(training_calendar_path, sep=',', engine='python')
df_product_train = pd.read_csv(training_product_details_path, sep=';', engine='python')
df_promotion_train = pd.read_csv(training_promotion_details_path, sep=';', engine='python', decimal=',')
df_transactions_train = pd.read_csv(training_transactions_path, sep=';', engine='python', decimal=',')
df_weather_train = pd.read_csv(training_weather_path, sep=',', engine='python')

# Importing forecasting data
df_calendar_forecast = pd.read_csv(forecast_calendar_path, sep=';', engine='python')
df_product_forecast = pd.read_csv(forecast_product_details_path, sep=';', engine='python')
df_promotion_forecast = pd.read_csv(forecast_promotion_details_path, sep=';', engine='python')
df_transactions_forecast = pd.read_csv(forecast_transactions_path, sep=';', engine='python')


In [8]:
mapping = {'J': 1, 'N': 0}

df_calendar_train['date'] = pd.to_datetime(df_calendar_train['date'])
df_calendar_train['holidayEventIndicator'] = df_calendar_train['holidayEventIndicator'].map(mapping)
df_calendar_train['workingDayIndicator'] = df_calendar_train['workingDayIndicator'].map(mapping)
df_calendar_train = df_calendar_train.convert_dtypes()

df_calendar_forecast['date'] = pd.to_datetime(df_calendar_forecast['date'])
df_calendar_forecast['holidayEventIndicator'] = df_calendar_forecast['holidayEventIndicator'].map(mapping)
df_calendar_forecast['workingDayIndicator'] = df_calendar_forecast['workingDayIndicator'].map(mapping)
df_calendar_forecast = df_calendar_forecast.convert_dtypes()

In [9]:
# Add lag/lead suffixes to holidayEventName
# For consecutive holidays: use EARLIER holiday for lags, LATER holiday for lead

def add_holiday_suffixes(df):
    df = df.sort_values('date').reset_index(drop=True)
    holidays = df[df['holidayEventIndicator'] == 1]['date'].values
    
    # For each date, find nearest upcoming/past holiday
    for idx, date in enumerate(df['date']):
        if df.loc[idx, 'holidayEventIndicator'] == 1:
            continue  # Already a holiday
        
        upcoming = holidays[holidays > date]
        if len(upcoming) > 0:
            days_before = (min(upcoming) - date).days
            if 1 <= days_before <= 5:
                holiday_name = df[df['date'] == min(upcoming)]['holidayEventName'].iloc[0]
                df.loc[idx, 'holidayEventName'] = f'{holiday_name}_lag_{days_before}' if pd.notna(holiday_name) else f'lag_{days_before}'
                continue
        
        past = holidays[holidays < date]
        if len(past) > 0:
            if (date - max(past)).days == 1:
                holiday_name = df[df['date'] == max(past)]['holidayEventName'].iloc[0]
                df.loc[idx, 'holidayEventName'] = f'{holiday_name}_lead_1' if pd.notna(holiday_name) else 'lead_1'
    
    return df

df_calendar_train = add_holiday_suffixes(df_calendar_train)
df_calendar_forecast = add_holiday_suffixes(df_calendar_forecast)

In [10]:
df_calendar_train

,date,holidayEventIndicator,workingDayIndicator,holidayEventName,doWName
0,2021-01-01,1,0,Nieuwjaar,Vrijdag
1,2021-01-02,0,0,Nieuwjaar_lead_1,Zaterdag
2,2021-01-03,0,0,<NA>,Zondag
3,2021-01-04,0,1,<NA>,Maandag
4,2021-01-05,0,1,<NA>,Dinsdag
...,...,...,...,...,...
1456,2024-12-27,0,1,Tweede Kerstdag_lead_1,Vrijdag
1457,2024-12-28,0,0,<NA>,Zaterdag
1458,2024-12-29,0,0,<NA>,Zondag
1459,2024-12-30,0,1,<NA>,Maandag


In [11]:
df_calendar_train.info()
df_calendar_forecast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   1461 non-null   datetime64[ns]
 1   holidayEventIndicator  1461 non-null   Int64         
 2   workingDayIndicator    1461 non-null   Int64         
 3   holidayEventName       315 non-null    string        
 4   doWName                1461 non-null   string        
dtypes: Int64(2), datetime64[ns](1), string(2)
memory usage: 60.1 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   31 non-null     datetime64[ns]
 1   holidayEventIndicator  31 non-null     Int64         
 2   workingDayIndicator    31 non-null     Int64         
 3   holidayEve

In [12]:
df_product_train = df_product_train.convert_dtypes()
df_product_train.info()
df_product_forecast = df_product_forecast.convert_dtypes()
df_product_forecast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   articleId         248 non-null    string
 1   category          248 non-null    string
 2   mainProductGroup  248 non-null    string
dtypes: string(3)
memory usage: 5.9 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   articleId         248 non-null    string
 1   category          248 non-null    string
 2   mainProductGroup  248 non-null    string
dtypes: string(3)
memory usage: 5.9 KB


In [13]:
# Renaming weather dataframe
rename_map = {
    'STN':'station_id',
    'YYYYMMDD':'date',
    'DDVEC':'mean_wind_dir',
    'FHVEC':'mean_wind_speed_vec',
    'FG':'mean_wind_speed',
    'FHX':'max_hourly_wind',
    'FHXH':'hour_max_wind',
    'FHN':'min_hourly_wind',
    'FHNH':'hour_min_wind',
    'FXX':'max_gust',
    'FXXH':'hour_max_gust',
    'TG':'mean_temp',
    'TN':'min_temp',
    'TNH':'hour_min_temp',
    'TX':'max_temp',
    'TXH':'hour_max_temp',
    'T10N':'min_temp_10cm',
    'T10NH':'hour_min_temp_10cm',
    'SQ':'sunshine_hours',
    'SP':'sunshine_percent',
    'Q':'solar_radiation',
    'DR':'precip_duration',
    'RH':'rel_humidity',
    'RHX':'max_rel_humidity',
    'RHXH':'hour_max_rel_humidity',
    'PG':'mean_pressure',
    'PX':'max_pressure',
    'PXH':'hour_max_pressure',
    'PN':'min_pressure',
    'PNH':'hour_min_pressure',
    'VVN':'min_visibility',
    'VVNH':'hour_min_visibility',
    'VVX':'max_visibility',
    'VVXH':'hour_max_visibility',
    'NG':'cloud_cover',
    'UG':'mean_abs_humidity',
    'UX':'max_abs_humidity',
    'UXH':'hour_max_abs_humidity',
    'UN':'min_abs_humidity',
    'UNH':'hour_min_abs_humidity',
    'EV24':'evaporation_24h'
}

df_weather_train = df_weather_train.rename(columns=lambda c: rename_map.get(c.strip(), c.strip())
)

list(df_weather_train)

['station_id',
 'date',
 'mean_wind_dir',
 'mean_wind_speed_vec',
 'mean_wind_speed',
 'max_hourly_wind',
 'hour_max_wind',
 'min_hourly_wind',
 'hour_min_wind',
 'max_gust',
 'hour_max_gust',
 'mean_temp',
 'min_temp',
 'hour_min_temp',
 'max_temp',
 'hour_max_temp',
 'min_temp_10cm',
 'hour_min_temp_10cm',
 'sunshine_hours',
 'sunshine_percent',
 'solar_radiation',
 'precip_duration',
 'rel_humidity',
 'max_rel_humidity',
 'hour_max_rel_humidity',
 'mean_pressure',
 'max_pressure',
 'hour_max_pressure',
 'min_pressure',
 'hour_min_pressure',
 'min_visibility',
 'hour_min_visibility',
 'max_visibility',
 'hour_max_visibility',
 'cloud_cover',
 'mean_abs_humidity',
 'max_abs_humidity',
 'hour_max_abs_humidity',
 'min_abs_humidity',
 'hour_min_abs_humidity',
 'evaporation_24h']

In [14]:
df_weather_train['date'] = pd.to_datetime(df_weather_train['date'].astype(str), format='%Y%m%d')
df_weather_train = df_weather_train[(df_weather_train['date'].dt.year >= 2021) & (df_weather_train['date'].dt.year <= 2024)]
for col in df_weather_train.select_dtypes(include='object').columns:
    df_weather_train[col] = pd.to_numeric(df_weather_train[col], errors='coerce')

keep_cols = [
    'date',
    'max_temp',
    'min_temp',
    'sunshine_hours',
    'precip_duration',
    'mean_wind_speed',
    'max_gust'
]

df_weather_train = df_weather_train[keep_cols]

df_weather_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1461 entries, 43830 to 45290
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             1461 non-null   datetime64[ns]
 1   max_temp         1461 non-null   int64         
 2   min_temp         1461 non-null   int64         
 3   sunshine_hours   1461 non-null   int64         
 4   precip_duration  1461 non-null   int64         
 5   mean_wind_speed  1461 non-null   int64         
 6   max_gust         1461 non-null   int64         
dtypes: datetime64[ns](1), int64(6)
memory usage: 91.3 KB


In [15]:
# Compute weather features correlation matrix
weather_corr = df_weather_train.drop(columns=['date']).corr()
weather_corr


,max_temp,min_temp,sunshine_hours,precip_duration,mean_wind_speed,max_gust
max_temp,1.000000,0.827809,0.557742,-0.247681,-0.234985,-0.131531
min_temp,0.827809,1.000000,0.166727,-0.025799,-0.004784,0.052701
sunshine_hours,0.557742,0.166727,1.000000,-0.469880,-0.236910,-0.197223
precip_duration,-0.247681,-0.025799,-0.469880,1.000000,0.356230,0.388700
mean_wind_speed,-0.234985,-0.004784,-0.236910,0.356230,1.000000,0.870087
max_gust,-0.131531,0.052701,-0.197223,0.388700,0.870087,1.000000


In [16]:
# Map row with the problematic value to null, then handled with all other nulls later
df_promotion_train.loc[ df_promotion_train['discountPct'] == "1.000.000.003", 'discountPct'] = pd.NA

df_promotion_train['discountPct'] = pd.to_numeric(df_promotion_train['discountPct'])
# NOTE: there are some null discountPct - keep for now and study in the data analysis file
df_promotion_train = df_promotion_train.convert_dtypes()
df_promotion_train.info()

df_promotion_forecast['discountPct'] = pd.to_numeric(df_promotion_forecast['discountPct'])
df_promotion_forecast = df_promotion_forecast.convert_dtypes()
df_promotion_forecast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6576 entries, 0 to 6575
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   promo_id     6576 non-null   string 
 1   articleId    6576 non-null   string 
 2   discountPct  6290 non-null   Float64
dtypes: Float64(1), string(2)
memory usage: 160.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   promo_id     247 non-null    string 
 1   articleId    247 non-null    string 
 2   discountPct  247 non-null    Float64
dtypes: Float64(1), string(2)
memory usage: 6.2 KB


In [17]:
df_transactions_train['date']  = pd.to_datetime(df_transactions_train['date'], format='%d-%m-%Y')
df_transactions_train = df_transactions_train.convert_dtypes()
df_transactions_train.info()

df_transactions_forecast['date']  = pd.to_datetime(df_transactions_forecast['date'], format='%d-%m-%Y')
df_transactions_forecast = df_transactions_forecast.convert_dtypes()
df_transactions_forecast.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343849 entries, 0 to 343848
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   date                343849 non-null  datetime64[ns]
 1   articleId           343849 non-null  string        
 2   storeCount          343849 non-null  Int64         
 3   FSC_index           343831 non-null  Float64       
 4   sales_volume_index  343849 non-null  Float64       
 5   promo_id            46012 non-null   string        
dtypes: Float64(2), Int64(1), datetime64[ns](1), string(2)
memory usage: 16.7 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7579 entries, 0 to 7578
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                7579 non-null   datetime64[ns]
 1   articleId           7579 non-null   string        
 2   storeCount        

Merging the dataframes into one

In [18]:
df_master_train = df_transactions_train.copy()

df_master_train = df_master_train.merge(
    df_promotion_train,
    on=['promo_id', 'articleId'],
    how='left',
    validate='m:1' 
)

df_master_train = df_master_train.merge(
    df_product_train,
    left_on='articleId',
    right_on='articleId',
    how='left',
    validate='m:1'
)

df_master_train = df_master_train.merge(
    df_calendar_train,
    on='date',
    how='left',
    validate='m:1'
)

df_master_train = df_master_train.merge(
    df_weather_train,
    on='date',
    how='left',
    validate='m:1'
)


df_master_forecast = df_transactions_forecast.copy()

df_master_forecast = df_master_forecast.merge(
    df_promotion_forecast,
    on=['promo_id', 'articleId'],
    how='left',
    validate='m:1' 
)

df_master_forecast = df_master_forecast.merge(
    df_product_forecast,
    left_on='articleId',
    right_on='articleId',
    how='left',
    validate='m:1'
)

df_master_forecast = df_master_forecast.merge(
    df_calendar_forecast,
    on='date',
    how='left',
    validate='m:1'
)


In [19]:
df_master_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343849 entries, 0 to 343848
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   date                   343849 non-null  datetime64[ns]
 1   articleId              343849 non-null  string        
 2   storeCount             343849 non-null  Int64         
 3   FSC_index              343831 non-null  Float64       
 4   sales_volume_index     343849 non-null  Float64       
 5   promo_id               46012 non-null   string        
 6   discountPct            43964 non-null   Float64       
 7   category               343849 non-null  string        
 8   mainProductGroup       343849 non-null  string        
 9   holidayEventIndicator  343849 non-null  Int64         
 10  workingDayIndicator    343849 non-null  Int64         
 11  holidayEventName       72240 non-null   string        
 12  doWName                343849 non-null  stri

In [20]:
df_master_train.head()

,date,articleId,storeCount,FSC_index,sales_volume_index,promo_id,discountPct,category,mainProductGroup,holidayEventIndicator,workingDayIndicator,holidayEventName,doWName,max_temp,min_temp,sunshine_hours,precip_duration,mean_wind_speed,max_gust
0,2021-01-01,0112d194d01727f5f5ba3c835c9ef20b76a3432d74f771...,619,100.0,100.0,<NA>,<NA>,"Vlees, Kip en Vis",Vlees,1,0,Nieuwjaar,Vrijdag,57,-35,9,0,21,60
1,2021-01-01,030c9c75de06d0aee298db030bf76b1933bb5a9a28e3fb...,560,100.0,100.0,<NA>,<NA>,"Vlees, Kip en Vis",Vis,1,0,Nieuwjaar,Vrijdag,57,-35,9,0,21,60
2,2021-01-01,03a945e32405653c15915432f70ca7ec3b3709fe6abf28...,620,100.0,100.0,<NA>,<NA>,Zoetwaren,chocolade,1,0,Nieuwjaar,Vrijdag,57,-35,9,0,21,60
3,2021-01-01,05f353ef4d503dd53818830120f4f479815b22d5ecc1c5...,317,100.0,100.0,<NA>,<NA>,"Vlees, Kip en Vis",Visconserven,1,0,Nieuwjaar,Vrijdag,57,-35,9,0,21,60
4,2021-01-01,06e708c215c11fe45c8b08104348be1cbdbd7ab7c1bf3e...,620,100.0,100.0,<NA>,<NA>,Wijn en PSV,Wit fris&fruitig,1,0,Nieuwjaar,Vrijdag,57,-35,9,0,21,60


In [21]:
# Save merged dataframes to CSV files
merged_data_path = os.path.join(os.getcwd(), 'data', 'merged_data')
os.makedirs(merged_data_path, exist_ok=True)

master_train_path = os.path.join(merged_data_path, 'master_train.csv')
df_master_train.to_csv(master_train_path, index=False)

master_forecast_path = os.path.join(merged_data_path, 'master_forecast.csv')
df_master_forecast.to_csv(master_forecast_path, index=False)

In [1]:
a=456
print(a)

456
